In [1]:

import pandas as pd
import plotly.express as px

# The full data set is ~30 Mb so this might not be fast...
# Grab the last year of the data
data = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/master/DataSets/omahaNOAA.csv")
# Clean it up
data = data.loc[len(data)-365*24:, ['DATE', 'HOURLYDRYBULBTEMPC']]
data.columns = ['date', 'temp_c']
data = data.loc[data['temp_c']!=0] # temp=0 is a 'missing value', which is annoying but fixable
data['date'] = pd.to_datetime(data['date'])
# And plot it
px.scatter(data, x='date', y='temp_c')

In [2]:
px.scatter(data[-100:], x='date', y='temp_c')

In [3]:
import pandas as pd, numpy as np
import statsmodels.api as sm
import statsmodels.tsa.stattools as st
import plotly.express as px
from datetime import datetime

# Collect data, set index

data = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/master/DataSets/pollutionBeijing.csv")
                   
format = '%Y-%m-%d %H:%M:%S'
data['datetime'] = pd.to_datetime(data['datetime'], format=format)
data.set_index(pd.DatetimeIndex(data['datetime']), inplace=True)

# Select variables for VAR model
varData = data[['pm2.5','TEMP','PRES','Iws']].dropna()[:-50]

In [4]:
model = sm.tsa.VAR(varData) # define the model and data
print(model.select_order().summary()) # uses information criteria to select the model order

C:\Users\cmisl\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        25.21       25.21   8.841e+10       25.21
1        11.97       11.97   1.578e+05       11.97
2        11.73       11.74   1.247e+05       11.74
3        11.59       11.60   1.079e+05       11.59
4        11.55       11.57   1.040e+05       11.56
5        11.54       11.56   1.029e+05       11.55
6        11.53       11.55   1.014e+05       11.53
7        11.52       11.54   1.003e+05       11.52
8        11.50       11.53   9.889e+04       11.51
9        11.48       11.51   9.705e+04       11.49
10       11.46       11.49   9.481e+04       11.47
11       11.43       11.47   9.206e+04       11.44
12       11.40       11.44   8.926e+04       11.41
13       11.37       11.42   8.690e+04       11.39
14       11.35       11.40   8.480e+04       11.36
15       11.32       11.37   8.268e+04       11.34
16       11.30       11.35   8.

In [5]:
modelFit = model.fit(48) 
modelFit.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 03, Feb, 2025
Time:                     17:04:47
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                    11.2901
Nobs:                     41659.0    HQIC:                   11.1806
Log likelihood:          -467508.    FPE:                    68190.3
AIC:                      11.1301    Det(Omega_mle):         66941.2
--------------------------------------------------------------------
Results for equation pm2.5
               coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------
const             4.088929        27.508743            0.149           0.882
L1.pm2.5          1.127753         0.004923          229.077           0.000
L1.TEMP          -0.837321         0.097215           -8.613           0.000

In [6]:
forecastData = data[['pm2.5','TEMP','PRES','Iws']].dropna()[-100:-50]

fcast = pd.DataFrame(modelFit.forecast(y = forecastData.values,steps=50), columns = ['pm2.5','TEMP','PRES','Iws'])

In [7]:
truth = data[['pm2.5','TEMP','PRES','Iws']].dropna()[-50:]

# Create and format the figure
fig = px.line(x = truth.index, 
		y=[fcast['pm2.5'], truth['pm2.5']],
        title = 'Particulate Matter Forecast',
		labels = {
			'value' : 'Particulate Matter',
			'x' : 'Date',
			'variable' : 'Series'
		})

# Renaming the series
fig.data[0].name = "Forecast"
fig.data[1].name = "Truth"

# Render the plot
fig.show()
     

In [8]:
# Create and format the figure
fig = px.line(x = truth.index, 
		y=[fcast['Iws'], truth['Iws']],
        title = 'Wind Speed Forecast',
		labels = {
			'value' : 'Wind Speed',
			'x' : 'Date',
			'variable' : 'Series'
		})

# Renaming the series
fig.data[0].name = "Forecast"
fig.data[1].name = "Truth"

# Render the plot
fig.show()
     

In [9]:

# Create and format the figure
fig = px.line(x = truth.index, 
		y=[fcast['TEMP'], truth['TEMP']],
        title = 'Temperature Forecast',
		labels = {
			'value' : 'Temperature (C)',
			'x' : 'Date',
			'variable' : 'Series'
		})

# Renaming the series
fig.data[0].name = "Forecast"
fig.data[1].name = "Truth"

# Render the plot
fig.show()

In [10]:
# Create and format the figure
fig = px.line(x = truth.index, 
		y=[fcast['PRES'], truth['PRES']],
        title = 'Air Pressure Forecast',
		labels = {
			'value' : 'Air Pressure',
			'x' : 'Date',
			'variable' : 'Series'
		})

# Renaming the series
fig.data[0].name = "Forecast"
fig.data[1].name = "Truth"

# Render the plot
fig.show()

In [ ]:
# Prep our data
varData = data[['pm2.5','TEMP','PRES','Iws', 'DEWP']].dropna()[-500:-50]
exog = varData['DEWP']
varData.drop('DEWP', axis=1, inplace=True)

# Create and fit model
model = sm.tsa.VARMAX(endog = varData.values, exog=exog.values, order=(48,0)) # define the order here for VARMAX!
modelFit = model.fit() 
modelFit.summary()
     

C:\Users\cmisl\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:374: UserWarning:

Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

